<a href="https://colab.research.google.com/github/busranur-sr/RNN/blob/main/rnn_with_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import nltk
import pandas as pd
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [2]:
#Loading the dataset
data = pd.read_csv('Emotion_final.csv',on_bad_lines='skip')
# Creating a new column sentiment based on overall ratings
data.isnull().sum()
def cleaning(df, stop_words):
    
    df['Text'] = df['Text'].apply(lambda x:' '.join(x.lower() for x in x.split()))
    # Replacing the special characters

    #df['verified_reviews'] = df['verified_reviews'].str.replace('[^ws]', '')
    # Replacing the digits/numbers

    df['Text'] = df['Text'].str.replace('d', '')

    # Removing stop words

    df['Text'] = df['Text'].apply(lambda x:' '.join(x for x in x.split() if x not in stop_words))

    # Lemmatization
    df['Text'] = df['Text'].apply(lambda x:' '.join([Word(x).lemmatize() for x in x.split()]))
    return df

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
data_v1 = cleaning(data, stop_words)
data_v1

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,Text,Emotion
0,int feel humiliate,sadness
1,go feeling hopeless amne hopeful aroun someone...,sadness
2,im grabbing minute post feel greey wrong,anger
3,ever feeling nostalgic fireplace know still pr...,love
4,feeling grouchy,anger
...,...,...
9890,go church probably sit back feel awkwar talk a...,sadness
9891,feel suspicious wrinkle prevention beauty prou...,fear
9892,starting feel emotional,sadness
9893,use vegetable glycerin oil cleansing mixture n...,surprise


In [3]:
# Encoded the target column
lb=LabelEncoder()
data_v1['Emotion'] = lb.fit_transform(data_v1['Emotion'])
data_v1

,Text,Emotion
0,int feel humiliate,4
1,go feeling hopeless amne hopeful aroun someone...,4
2,im grabbing minute post feel greey wrong,0
3,ever feeling nostalgic fireplace know still pr...,3
4,feeling grouchy,0
...,...,...
9890,go church probably sit back feel awkwar talk a...,4
9891,feel suspicious wrinkle prevention beauty prou...,1
9892,starting feel emotional,4
9893,use vegetable glycerin oil cleansing mixture n...,5


In [17]:
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data_v1['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X)
vocab_size = 500
X[1:10]
output_size = 5

In [18]:
hidden_size = 100
#seq_length = 25
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
Why = np.random.randn(output_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((output_size, 1))

In [ ]:
import numpy as np
def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  #store our inputs, hidden states, outputs, and probability values
  
  for t in reversed(range(len(inputs))):
    #output probabilities
    dy = np.copy(ps[t])
    #derive our first gradient
    dy[targets[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(Whh.T, dhraw) 
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [21]:
data = open('kafka.txt', 'r').read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print ('data has %d chars, %d unique' % (data_size, vocab_size))
vocab_size

data has 137628 chars, 80 unique


80

In [22]:
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print (char_to_ix)
print (ix_to_char)

{'R': 0, '9': 1, '2': 2, 'K': 3, '!': 4, '6': 5, 'A': 6, 'S': 7, 'P': 8, 'Y': 9, 'X': 10, '\n': 11, 'q': 12, 'g': 13, 'V': 14, 'J': 15, 'ç': 16, 'o': 17, 'N': 18, 'Q': 19, 'x': 20, 'k': 21, ':': 22, 'l': 23, 'M': 24, 'T': 25, 'D': 26, '(': 27, 'h': 28, 'O': 29, 's': 30, 't': 31, '$': 32, 'W': 33, '*': 34, 'j': 35, 'u': 36, 'w': 37, 'C': 38, ';': 39, 'H': 40, '%': 41, "'": 42, '8': 43, '@': 44, 'i': 45, 'v': 46, '1': 47, '4': 48, 'm': 49, 'n': 50, '"': 51, 'I': 52, '3': 53, '.': 54, 'B': 55, ')': 56, 'f': 57, '7': 58, 'd': 59, 'L': 60, 'c': 61, 'G': 62, 'E': 63, 'F': 64, ' ': 65, 'e': 66, '0': 67, '-': 68, 'p': 69, 'a': 70, 'U': 71, 'y': 72, 'z': 73, ',': 74, '5': 75, 'b': 76, '?': 77, '/': 78, 'r': 79}
{0: 'R', 1: '9', 2: '2', 3: 'K', 4: '!', 5: '6', 6: 'A', 7: 'S', 8: 'P', 9: 'Y', 10: 'X', 11: '\n', 12: 'q', 13: 'g', 14: 'V', 15: 'J', 16: 'ç', 17: 'o', 18: 'N', 19: 'Q', 20: 'x', 21: 'k', 22: ':', 23: 'l', 24: 'M', 25: 'T', 26: 'D', 27: '(', 28: 'h', 29: 'O', 30: 's', 31: 't', 32: '$',

In [23]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print (vector_for_char_a.ravel())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [66]:
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [89]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print ("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print ("targets", targets)

inputs [29, 50, 66, 65, 49, 17, 79, 50, 45, 50, 13, 74, 65, 37, 28, 66, 50, 65, 62, 79, 66, 13, 17, 79, 65]
targets [50, 66, 65, 49, 17, 79, 50, 45, 50, 13, 74, 65, 37, 28, 66, 50, 65, 62, 79, 66, 13, 17, 79, 65, 7]


In [78]:
hprev = [[0]]*100
hprev

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0]]

In [91]:
import numpy as np
def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  #store our inputs, hidden states, outputs, and probability values
  xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
  xs, hs, ys, ps = {}, {}, {}, {}
  #init with previous hidden state
    # Using "=" would create a reference, this creates a whole separate copy
    # We don't want hs[-1] to automatically change if hprev is changed
  hs[-1] = np.copy(hprev)
  #init loss as 0
  loss = 0
  # forward pass                                                                                                                                                                              
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
    xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
  ys[1] = np.dot(Why, hs[len(inputs)-1]) + by # unnormalized log probabilities for next chars                                                                                                            
  ps[1] = np.exp(ys[1]) / np.sum(np.exp(ys[1])) # probabilities for next chars                                                                                                              
  loss += -np.log(ps[1][targets[len(inputs)-1],0]) # softmax (cross-entropy loss)                                                                                                                       
    # backward pass: compute gradients going backwards    
     #initalize vectors for gradient values for each set of weights 
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why) 
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])

  for t in reversed(range(len(inputs))):
    t = len(inputs)-1
    #output probabilities
    dy = np.copy(ps[1])
    #derive our first gradient
    dy[targets[len(inputs)-1]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(Whh.T, dhraw) 
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]



In [88]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step   
  n is how many characters to predict
  """
  #create vector
  x = np.zeros((vocab_size, 1))
  #customize it for our seed char
  x[seed_ix] = 1
  #list to store generated chars
  ixes = []
  #for as many characters as we want to generate
  for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    #compute output (unnormalised)
    y = np.dot(Why, h) + by
    ## probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
    x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
    x[ix] = 1
    #add it to the list
    ixes.append(ix)

  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print ('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)

----
 Nb0vg,Yamçd:zGçMn)aoO$4N4;?G**E*dmmJEae@*1'.YEvE@Sefj;LL4Ve:XUFJPv?ggc
VG2g,iyLFvW*w!.;gVjJ)4I,XL):
BIVçTg9mUu%91xhç"AnUh3WnvRoçAcholrTiwwC@,R,sd2n4O$JWXT"-;uB6Wy)F@:0oaV?@jGD aTL-Kc(FQi.Q:%r0FTz4çIWl 
----


In [ ]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                         
  n += 1 # iteration counter    